<a href="https://colab.research.google.com/github/ded-mikhalych/TK_SMM_LRs/blob/main/%D0%9B%D0%A04_%D0%A1%D0%B0%D1%84%D0%BE%D0%BD%D0%BE%D0%B2%D0%B0_%D0%9F_%D0%94_%2C_%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B5%D0%BD%D0%BA%D0%BE%D0%B2_%D0%9A_%D0%95_%2C_%D0%9C%D0%B8%D1%85%D0%B0%D0%BB%D1%8C%D1%87%D1%83%D0%BA_%D0%94_%D0%90_%2C_%D0%9E%D0%B4%D0%B8%D0%BD_%D0%92_%D0%95_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rnd
import math

### 4.1 Написать функцию формирования порождающей и проверочной матриц расширенного кода Голея (24, 12, 8)

In [ ]:
def get_G_H_matrix():
    b_matr = get_B_matr()
    eye_matr = np.eye(b_matr.shape[0], dtype=int)
    return np.hstack((eye_matr, b_matr)), np.vstack((eye_matr, b_matr))

In [ ]:
def get_B_matr():
    row = [1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0]
    matrix = [row]
    while len(matrix) != 11:
        deleted = row.pop(0)
        row.append(deleted)
        matrix.append(row.copy())
    eye_vec = np.ones((11, 1), dtype=int)
    matrix = np.concatenate((matrix, eye_vec), axis=1)
    eye_vec = np.ones((1, 12), dtype=int)
    matrix = np.concatenate((matrix, eye_vec), axis=0)
    matrix[-1][-1] = 0
    return matrix

In [ ]:
g_matr, h_matr = get_G_H_matrix()

In [ ]:
g_matr

In [ ]:
h_matr

### 4.2. Провести исследование расширенного кода Голея для одно-, двух-, трёх- и четырёхкратных ошибок

In [ ]:
def get_err_word(g_matr, h_matr, t, iter_max = 25):
    row = g_matr.shape[0]
    u = np.zeros((1, 24), dtype=int)
    w_err = np.zeros((1, 24), dtype=int)

    iterations = 0
    while True and iterations <= iter_max:
        idx = rnd.randint(0, row - 1)
        word = np.array(g_matr[idx][:row])
        w = np.dot(word, g_matr) % 2
        print(f"Исходное сообщение: {word}")
        print(f"Отправленное сообщение: {w}")
        w_err = w.copy()
        for i in range(t):
            w_err[i] += 1
            w_err[i] %= 2
        print(f"Принятое сообщение с ошибкой: {w_err}")
        s = np.dot(w_err, h_matr) % 2
        print(f"Синдром: {s}")
        b_matr = get_B_matr()
        sB = np.dot(s, b_matr) % 2
        e = np.zeros((1, 12), dtype=int)
        if sum(s) <= 3:
            u = hconcat(s, e)
            break
        elif get_rows_sum_B(s, b_matr)[0]:
            rows_sum_B_res = get_rows_sum_B(s, b_matr)
            e[0][rows_sum_B_res[1]] = 1
            u = hconcat(rows_sum_B_res[2], e)
            break
        elif sum(sB) <= 3:
            u = hconcat(e, sB)
            break
        elif get_rows_sum_B(sB, b_matr)[0]:
            rows_sum_B_res = get_rows_sum_B(sB, b_matr)
            e[0][rows_sum_B_res[1]] = 1
            u = hconcat(e, rows_sum_B_res[2])
            break
        else:
            print("Ошибка не найдена. Запускаю повторный цикл\n")
            iterations += 1
    if iterations <= iter_max:
        print(f"Ошибка: {u}")
        fixed_message = (w_err + u) % 2
        print(f"Исправленное сообщение: {fixed_message}")
        print(f"Декодированное сообщение: {fixed_message[0][:row]}")
    else:
        print("Ошибка: превышен лимит итераций")

In [ ]:
def get_rows_sum_B(s, b_matr):
    for i in range(b_matr.shape[0]):
        rows_sum = (s + b_matr[i]) % 2
        if sum(rows_sum) <= 2:
            return True, i, rows_sum
    return False, -1, None

In [ ]:
def hconcat(matr1, matr2):
    if matr1.ndim == 1:
        matr1 = np.array([matr1])
    if matr2.ndim == 1:
        matr2 = np.array([matr2])
    return np.hstack((matr1, matr2))

In [ ]:
t_list = [1, 2, 3, 4]

In [ ]:
for t in t_list:
    print(f"\033[1mИсследование расширенного кода Голея при t = {t}\033[0m")
    get_err_word(g_matr, h_matr, t)
    if (t != t_list[-1]):
        print("\n")

### 4.3 Написать функцию формирования порождающей и проверочных матриц кода Рида-Маллера RM(r, m) на основе параметров r и m

In [ ]:
def get_rm_G_matr(r, m):
    if r == 0:
        return np.ones(2 ** m, dtype=int)
    if r == m:
        z = np.zeros(2 ** m, dtype=int)
        z[-1] = 1
        return np.vstack((get_rm_G_matr(m - 1, m), z))
    upper_m = get_rm_G_matr(r, m - 1)
    right_l_corner_m = get_rm_G_matr(r - 1, m - 1)
    zeros = np.zeros(right_l_corner_m.shape, dtype=int)
    matrix1 = np.vstack((upper_m, zeros))
    matrix2 = np.vstack((upper_m, right_l_corner_m))
    return np.hstack((matrix1, matrix2))

In [ ]:
def get_rm_H_matr(i, m):
    h_matr = [[1, 1], [1, -1]]
    I_1 = np.eye(2 ** (m - i), dtype=int)
    I_2 = np.eye(2 ** (i - 1), dtype=int)
    # np.kron - произведение Кронекера
    return np.kron(np.kron(I_1, h_matr), I_2)

### 4.4. Провести исследование кода Рида-Маллера RM(1,3) для одно- и двукратных ошибок

In [ ]:
def get_rm_err_word(g_matr, t):
    m = int(math.log2(g_matr.shape[1]))
    row = g_matr.shape[0]

    idx = rnd.randint(0, row - 1)
    word = np.array(g_matr[idx][:row])
    w = np.dot(word, g_matr) % 2
    print(f"Исходное сообщение: {word}")
    print(f"Отправленное сообщение: {w}")
    for i in range(t):
        w[i] += 1
        w[i] %= 2
    print(f"Принятое сообщение с ошибкой: {w}")
    w_err = [-1 if elem == 0 else 1 for elem in w]
    print(f"Изменённое сообщение: {w_err}")
    for i in range(1, m + 1):
        w_err = np.dot(w_err, get_rm_H_matr(i, m))
    print(f"Изменённое сообщение после преобразования: {w_err}")
    idx = np.where(w_err == max(w_err, key=abs))[0][0]
    byte_len = len(f"{idx:b}")
    extra_zeros = "0" * (m - byte_len) if (m - byte_len) > 0 else ""
    byte_row = f"{extra_zeros}{idx:b}"[::-1]
    if w_err[idx] > 0:
        print(f'Декодированное сообщение: {list(map(lambda x: int(x), f"1{byte_row}"))}')
    else:
        print(f'Декодированное сообщение: {list(map(lambda x: int(x), f"0{byte_row}"))}')

In [ ]:
rm_g_matr = get_rm_G_matr(1, 3)

In [ ]:
rm_g_matr

In [ ]:
t_list_rm13 = [1, 2]

In [ ]:
for t in t_list_rm13:
    print(f"\033[1mИсследование расширенного кода Рида-Маллера RM(1, 3) при t = {t}\033[0m")
    get_rm_err_word(rm_g_matr, t)
    if (t != t_list_rm13[-1]):
        print("\n")

### 4.5. Провести исследование кода Рида-Маллера RM(1,4) для одно-, двух-, трёх- и четырёхкратных ошибок

In [ ]:
rm_g_matr_rm14 = get_rm_G_matr(1, 4)

In [ ]:
rm_g_matr_rm14

In [ ]:
t_list_rm14 = [1, 2, 3, 4]

In [ ]:
for t in t_list_rm14:
    print(f"\033[1mИсследование расширенного кода Рида-Маллера RM(1, 4) при t = {t}\033[0m")
    get_rm_err_word(rm_g_matr_rm14, t)
    if (t != t_list_rm14[-1]):
        print("\n")